# Appmode in Jupyter

This page demonstrates how to generate online "apps" with a Jupyter Notebook interface. With the `appmode` plugin, we can create interactive experiences without requiring coding or running a specific cell.

Markdown cells will be retained, and all code cells will be run, then hidden. The outputs of each cell will be displayed as well.

Check out a further example using an [ipyvolume](https://ipyvolume.readthedocs.io/en/latest/) to render a 3D plot: [ipyvolume_demo.ipynb](ipyvolume_demo.ipynb)

In [ ]:
from __future__ import print_function,division
import IPython
from IPython.display import Javascript,clear_output,display 
from ipywidgets import interact,interactive,fixed,interact_manual,Layout,jslink,Button,Layout,jslink,IntText,IntSlider,TwoByTwoLayout,IntSlider,Label,Text
import ipywidgets as widgets
import copy 
from bs4 import BeautifulSoup 
from urllib.request import Request, urlopen 
import pandas as pd 
import requests 
from io import StringIO 
from pychord import Chord

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

In [3]:
# import song list
def song_table():
    orig_url='https://drive.google.com/file/d/1aDrlFu5eSabjo8CVKAOG6ljfr5OihZLq/view?usp=sharing'
    file_id = orig_url.split('/')[-2]
    dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
    text = requests.get(dwn_url)
    text.encoding = 'utf-8'
    csv_raw = StringIO(text.text)
    df = pd.read_csv(csv_raw,encoding='utf-8')
    df['אמן'] = df['אמן'].apply(lambda x: x.strip())
    df['שיר'] = df['שיר'].apply(lambda x: x.strip())
    df['shir_aman'] = df['אמן']+" | "+df['שיר'] 
    return df

df = song_table()

In [4]:
def artist_songs(artist_name='א'):
      song_list = df.loc[df['shir_aman'].str.contains(artist_name)]['shir_aman'].sort_values()
      song_list = list(zip(song_list,song_list.index))
      return song_list

In [36]:
song_tab = widgets.Select(options=artist_songs(),rows=10,layout=Layout(width='auto'))

text = widgets.Text(value='',
                    placeholder='חפש שיר או אמן',
                    layout=Layout(width='auto'),
                    disabled=False)

def song_search_change(change):
    new_options = artist_songs(change.new)
    song_tab.options = new_options

text.observe(song_search_change, names='value')
#display(text,song_tab)

In [6]:
#song_list = df['shir_aman'].tolist()
#widgets.Combobox(
    # value='John',
 #   placeholder='Choose Someone',
  #  options=song_list,
   # description='Combobox:',
    #ensure_option=True,
    #disabled=False,
    #index = True
#)

Combobox(value='', description='Combobox:', ensure_option=True, options=('Christina Aguilera | Your Body', 'Ma…

In [37]:

button_LoadSong = widgets.Button(description="🤩 Get my Song 🎤",layout=Layout(width='auto'))
button_ChangeChords = widgets.Button(description="  Transform Chords  ",layout=Layout(width='auto'))
output = widgets.Output()
progression_picker = widgets.Select(options=[6,5,4,3,2,1,0,-1,-2,-3,-4,-5],value=0,rows=1,layout=Layout(width='auto'))

song_show = widgets.HTML(value='',layout=Layout(width='auto'))
song_show.layout.grid_auto_flow = 'row dense'
#song_show.layout.max_height = '200px'
#song_show.layout.grid_auto_columns = 'True'



display(text,song_tab,button_LoadSong,button_ChangeChords,progression_picker,song_show)

#display(button_LoadSong,button_ChangeChords,progression_picker,output)

Prog_value = progression_picker.value

def on_button_clicked(b):
    with output:
        #clear_output(True)
        #display(IPython.display.HTML("<marquee style='width: 30%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"))
        #clear_output(True)
        song_show.value = "<marquee style='width: 50%; color: purple;'><b>✌ ...טוען שיר</b></marquee>"
        myurl = df.iloc[song_tab.value]['לינק']
        req = Request(myurl, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soupified = BeautifulSoup(webpage,'lxml' ) # 'html.parser'
        song = soupified.find("div", {"class": "song_block"}) #Find the nevessary tag and class which it belongs to
        header = song.find(attrs={"class":"song_block_title"})
        # Clean the HTML from ads and more unnecessary stuff
        header = header.get_text().strip().replace("אקורדים לשיר ","")

        text_alignment = 'right'
        header = f"""<h1 style="text-align: {text_alignment}">"""+header+'</h1>'
        on_button_clicked.header = header

        progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
        header_progression = header+progression_status

        delete_nodes = [{"class":"song_rating"},
                        {"class":"row"},
                        {"id":"songad"},
                        {"class":"song_block_title"},
                        {"id":"songTopAd8Td"},
                        {"class":"videosTitle"},
                        {"type":"text/javascript"},
                        {"id":"taboola-below-article-thumbnails"},
                        {"id":"downInSongTable"},
                        {"class":"chord_info"}]

        for node in delete_nodes:
            for div in song.find_all(attrs=node): 
                div.decompose()

        # Align guitar TABS to the left
        tab_list = ''
        tab_nodes = [{"class":"tabs"}]
        for node in tab_nodes:
            for div in song.find_all(attrs=node):
                div['dir'] = 'ltr'
                div['align'] = 'left'
                #div['display'] = 'inline-block'
                #div['width'] = '300px'
                div['position'] = 'absolute'
                div['right'] = '10%'
                #div['margin-left'] = '300px'
                tab_list=tab_list+'<p'+str(div)+'</p>'
                
        on_button_clicked.song=song
        # Merge Header & Song(words+chords)
        full_page=header_progression+str(song)
        on_button_clicked.full_page = full_page
        song_show.value = full_page
        #display(IPython.display.HTML(full_page))
        return on_button_clicked.header,on_button_clicked.song



def Chord_Progression(orig_chord=str,transpose_level=0):
    prog_chord = Chord(orig_chord)
    prog_chord.transpose(transpose_level)
    return prog_chord.chord

def on_button_clicked1(b):
    with output:
        clear_output(True)
        Prog_value = progression_picker.value
        Progression_song = copy.copy(on_button_clicked.song)
        header = copy.copy(on_button_clicked.header)
        for elm in Progression_song.find_all('span'):
            try:
                  elm.string = elm.text.replace(elm.text, Chord_Progression(orig_chord=elm.text,transpose_level=Prog_value))
            except:
                  dummy = 0
        text_alignment = 'right'
        if Prog_value > 0:
            progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למעלה"+"</h4><h4></h4>"
            header_progression = header+progression_status
        elif Prog_value < 0:
            progression_status = f"""<h4 style="text-align: {text_alignment}">"""+f"מודולציה: {Prog_value*0.5} טון למטה"+"</h4><h4></h4>"
            header_progression = header+progression_status
        else:
            progression_status =f"""<h4 style="text-align: {text_alignment}">"""+f"הסולם המקורי"+"</h4><h4></h4>"
            header_progression = header+progression_status
      # Merge Header & Song(words+chords)
        full_page=header_progression+str(Progression_song)
        on_button_clicked1.full_page = full_page
        song_show.value = full_page
        #display(IPython.display.HTML(full_page))
        return on_button_clicked1.full_page
        #print(full_page)
        

    
display(Javascript(disable_js))
button_LoadSong.on_click(on_button_clicked)
button_ChangeChords.on_click(on_button_clicked1)

Text(value='', layout=Layout(width='auto'), placeholder='חפש שיר או אמן')

Select(layout=Layout(width='auto'), options=(('Adi Ulmansky | איך לאהוב אותי', 8703), ('Chesher | לפרק את מדינ…

Button(description='🤩 Get my Song 🎤', layout=Layout(width='auto'), style=ButtonStyle())

Button(description='  Transform Chords  ', layout=Layout(width='auto'), style=ButtonStyle())

Select(index=6, layout=Layout(width='auto'), options=(6, 5, 4, 3, 2, 1, 0, -1, -2, -3, -4, -5), rows=1, value=…

HTML(value='', layout=Layout(grid_auto_flow='row dense', width='auto'))

<IPython.core.display.Javascript object>

In [63]:
view = widgets.AppLayout(header=None,
                          left_sidebar=grid,
                          center=song_show,
                          right_sidebar=None,
                          footer=None,
                          height="100%", width="100%")

AppLayout(children=(GridspecLayout(children=(Text(value='', layout=Layout(grid_area='widget001', width='auto')…

In [62]:
#display(text,song_tab,button_LoadSong,button_ChangeChords,progression_picker,song_show)

grid = widgets.GridspecLayout(6, 1,height='30%',width='100%')
grid[0,0] = text
grid[1, 0] = song_tab
grid[2, 0] = button_LoadSong
grid[4, 0] = progression_picker
grid[5, 0] = button_ChangeChords
grid

GridspecLayout(children=(Text(value='', layout=Layout(grid_area='widget001', width='auto'), placeholder='חפש ש…

### widgets.HTML(value=sring)
song_show.layout.keys


In [127]:
sring = """<h1 style="text-align: right">לפרק את מדינת תל אביב - Chesher</h1><h4 style="text-align: right">הסולם המקורי</h4><h4></h4><div class="song_block"><a name="song"></a>
<tr>
</tr>
<tr>
</tr>
<tr>
</tr>
<tr>
</tr>
<div class="song_block_top">

<div class="song_block_content_wrap">
<div class="song_block_content" dir="rtl">
<div class="block_content_padd">
<table dir="ltr" width="100%">
<tr>

<td align="left" id="ytTd8" style="padding-left:5px;">
<div id="youtubeEMBD"></div>
</td>
</tr>
</table>
<span id="chordsArea"></span>

<div align="right" id="songContentTPL" style="padding:0px 5px 5px 5px;">
<table border="0" cellpadding="0" cellspacing="0">
<tbody>
<tr>
<td class="chords">
 <span onmouseout="document.getElementById('chord_1').style.display = 'none';" onmouseover="document.getElementById('chord_1').style.display = '';">E</span>      <span onmouseout="document.getElementById('chord_2').style.display = 'none';" onmouseover="document.getElementById('chord_2').style.display = '';">D</span>   <span onclick="RotateImgs('crimg_3', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_3').style.display = 'none';" onmouseover="document.getElementById('chord_3').style.display = '';" style="cursor:hand">G</span>            <span onmouseout="document.getElementById('chord_4').style.display = 'none';" onmouseover="document.getElementById('chord_4').style.display = '';">E</span>  
</td>
</tr>
<tr>
<td class="song">
המדינה בוערת אבל אני לא נשרף
</td>
</tr>
<tr>
<td class="chords">
 <span onmouseout="document.getElementById('chord_5').style.display = 'none';" onmouseover="document.getElementById('chord_5').style.display = '';">E</span>      <span onmouseout="document.getElementById('chord_6').style.display = 'none';" onmouseover="document.getElementById('chord_6').style.display = '';">D</span>   <span onclick="RotateImgs('crimg_7', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_7').style.display = 'none';" onmouseover="document.getElementById('chord_7').style.display = '';" style="cursor:hand">G</span>                   <span onmouseout="document.getElementById('chord_8').style.display = 'none';" onmouseover="document.getElementById('chord_8').style.display = '';">E</span>   
</td>
</tr>
<tr>
<td class="song">
יושב בתוך מקלט אטום, בחוץ עולם מטורף
</td>
</tr>
<tr>
<td class="chords">
<span onmouseout="document.getElementById('chord_9').style.display = 'none';" onmouseover="document.getElementById('chord_9').style.display = '';">B</span>         <span onclick="RotateImgs('crimg_10', Array('https://www.tab4u.com/additions/chords_imgs/A.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_A_2.gif'));" onmouseout="document.getElementById('chord_10').style.display = 'none';" onmouseover="document.getElementById('chord_10').style.display = '';" style="cursor:hand">A</span>      
</td>
</tr>
<tr>
<td class="song">
לא מביט כלל מסביב
</td>
</tr>
<tr>
<td class="chords">
<span onclick="RotateImgs('crimg_11', Array('https://www.tab4u.com/additions/chords_imgs/A.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_A_2.gif'));" onmouseout="document.getElementById('chord_11').style.display = 'none';" onmouseover="document.getElementById('chord_11').style.display = '';" style="cursor:hand">A</span>          <span onclick="RotateImgs('crimg_12', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_12').style.display = 'none';" onmouseover="document.getElementById('chord_12').style.display = '';" style="cursor:hand">G</span>    
</td>
</tr>
<tr>
<td class="song">
ממשיך ללכת בנתיב
</td>
</tr>
</tbody>
</table>
<br/><table border="0" cellpadding="0" cellspacing="0">
<tbody>
<tr>
<td class="chords">
 <span onmouseout="document.getElementById('chord_13').style.display = 'none';" onmouseover="document.getElementById('chord_13').style.display = '';">E</span>       <span onmouseout="document.getElementById('chord_14').style.display = 'none';" onmouseover="document.getElementById('chord_14').style.display = '';">D</span>       <span onclick="RotateImgs('crimg_15', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_15').style.display = 'none';" onmouseover="document.getElementById('chord_15').style.display = '';" style="cursor:hand">G</span>  <span onmouseout="document.getElementById('chord_16').style.display = 'none';" onmouseover="document.getElementById('chord_16').style.display = '';">E</span> 
</td>
</tr>
<tr>
<td class="song">
לפרק את מדינת תל אביב!
</td>
</tr>
</tbody>
</table>
<br/><table border="0" cellpadding="0" cellspacing="0">
<tbody>
<tr>
<td class="chords">
<span onmouseout="document.getElementById('chord_17').style.display = 'none';" onmouseover="document.getElementById('chord_17').style.display = '';">E</span>    <span onmouseout="document.getElementById('chord_18').style.display = 'none';" onmouseover="document.getElementById('chord_18').style.display = '';">D</span>   <span onclick="RotateImgs('crimg_19', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_19').style.display = 'none';" onmouseover="document.getElementById('chord_19').style.display = '';" style="cursor:hand">G</span>               <span onmouseout="document.getElementById('chord_20').style.display = 'none';" onmouseover="document.getElementById('chord_20').style.display = '';">E</span>   
</td>
</tr>
<tr>
<td class="song">
החולה לא מת אבל הנזק כבר נעשה
</td>
</tr>
<tr>
<td class="chords">
 <span onmouseout="document.getElementById('chord_21').style.display = 'none';" onmouseover="document.getElementById('chord_21').style.display = '';">E</span>  <span onmouseout="document.getElementById('chord_22').style.display = 'none';" onmouseover="document.getElementById('chord_22').style.display = '';">D</span>     <span onclick="RotateImgs('crimg_23', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_23').style.display = 'none';" onmouseover="document.getElementById('chord_23').style.display = '';" style="cursor:hand">G</span>           <span onmouseout="document.getElementById('chord_24').style.display = 'none';" onmouseover="document.getElementById('chord_24').style.display = '';">E</span> 
</td>
</tr>
<tr>
<td class="song">
והרופא בבית מנתח מהכורסה
</td>
</tr>
<tr>
<td class="chords">
<span onmouseout="document.getElementById('chord_25').style.display = 'none';" onmouseover="document.getElementById('chord_25').style.display = '';">B</span>         <span onclick="RotateImgs('crimg_26', Array('https://www.tab4u.com/additions/chords_imgs/A.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_A_2.gif'));" onmouseout="document.getElementById('chord_26').style.display = 'none';" onmouseover="document.getElementById('chord_26').style.display = '';" style="cursor:hand">A</span>    
</td>
</tr>
<tr>
<td class="song">
המצב עדיין יציב
</td>
</tr>
<tr>
<td class="chords">
<span onclick="RotateImgs('crimg_27', Array('https://www.tab4u.com/additions/chords_imgs/A.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_A_2.gif'));" onmouseout="document.getElementById('chord_27').style.display = 'none';" onmouseover="document.getElementById('chord_27').style.display = '';" style="cursor:hand">A</span>          <span onclick="RotateImgs('crimg_28', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_28').style.display = 'none';" onmouseover="document.getElementById('chord_28').style.display = '';" style="cursor:hand">G</span>        
</td>
</tr>
<tr>
<td class="song">
נושם לבד, אך לא מגיב
</td>
</tr>
</tbody>
</table>
<table border="0" cellpadding="0" cellspacing="0" class="br"><tbody><tr><td class="song" style="font-size: 14px;">מעבר:</td></tr><tr><td class="chords" style="font-size: 14px;">B A E E x4</td></tr></tbody></table><br/><table border="0" cellpadding="0" cellspacing="0">
<tbody>
<tr>
<td class="chords">
 <span onmouseout="document.getElementById('chord_29').style.display = 'none';" onmouseover="document.getElementById('chord_29').style.display = '';">E</span>       <span onmouseout="document.getElementById('chord_30').style.display = 'none';" onmouseover="document.getElementById('chord_30').style.display = '';">D</span>       <span onclick="RotateImgs('crimg_31', Array('https://www.tab4u.com/additions/chords_imgs/G.gif', 'https://www.tab4u.com/additions/chords_imgs/G_2.gif', 'https://www.tab4u.com/additions/chords_imgs/akord_G_3.gif'));" onmouseout="document.getElementById('chord_31').style.display = 'none';" onmouseover="document.getElementById('chord_31').style.display = '';" style="cursor:hand">G</span>  <span onmouseout="document.getElementById('chord_32').style.display = 'none';" onmouseover="document.getElementById('chord_32').style.display = '';">E</span> 
</td>
</tr>
<tr>
<td class="song">
לפרק את מדינת תל אביב!
</td>
</tr>
</tbody>
</table>
<br/><table border="0" cellpadding="0" cellspacing="0"><tbody><tr><td class="song" style="font-size: 14px;">סיום:</td></tr><tr><td class="chords" style="font-size: 14px;">E E B A x4</td></tr></tbody></table> </div>
<center>




</center></div>
</div>
</div>
</div>
</div>
"""